In [1]:
# Import the necessary libraries
from bs4 import BeautifulSoup
import requests
import csv
import urllib.parse

In [2]:

# Define the URL of the webpage you want to scrape
url = 'https://news.ycombinator.com/newest'

In [3]:
# Send a GET request to the URL
response = requests.get(url)

In [4]:

# Parse the HTML content of the webpage using Beautiful Soup
soup = BeautifulSoup(response.content, 'html.parser')


In [5]:
# Find all elements with a specific class
headlines = soup.find_all(class_='titleline')
sublines = soup.find_all(class_= 'subline')
more = soup.find('a', class_ = 'morelink')

In [6]:
# Fetch more news
for _ in range(3):
    href = more.get('href')
    
    # Parse the URL to extract query parameters
    parsed_url = urllib.parse.urlparse(href)
    
    # Get the query parameters as a dictionary
    query_params = urllib.parse.parse_qs(parsed_url.query)
    
    # Extract 'next' and 'n' parameters
    next_param = query_params.get('next')
    n_param = query_params.get('n')
    
    response = requests.get(url, params = {"n" : n_param, "next" : next_param})
    
    # Parse the HTML content of the webpage using Beautiful Soup
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find all elements with a specific class
    headlines += soup.find_all(class_='titleline')
    sublines += soup.find_all(class_= 'subline')
    more = soup.find('a', class_ = 'morelink')
    

In [7]:

# Specify the CSV file name
csv_file = "ycombinator_news.csv"
quantity = 100

# Write the data to CSV
with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["S. No.", "Headline", "Website", "point by", "minutes ago"])  # Write header row

    for i in range(quantity):
        
        headline = headlines[i].find('a').text
        website = "No website given" if (w := headlines[i].find('span', class_='sitestr')) is None else w.text
        point_by = sublines[i].find('a', class_= 'hnuser').text
        minutes_ago = sublines[i].find('span', class_= 'age').text
        writer.writerow([i + 1, headline, website, point_by, minutes_ago])

print("Data exported to", csv_file)


Data exported to ycombinator_news.csv
